# SPADL To Wyscout Conversion

### What is SPADL?

SPADL (Soccer Player Action Description Language) represents a game as a sequence of on-the-ball actions [a1, a2, ..., am], where m is the total number of actions that happened in the game.

SPADL uses a standardized coordinate system with the origin on the bottom left of the pitch, and a uniform field of 105m x 68m. For direction of play, SPADL uses the “home team attacks to the right” convention, but this can be converted conveniently with the play_left_to_right() function such that the lower x-coordinates represent the own half of the team performing the action.
<br>
<br>
**A note on Atomic SPADL**<br>
In this representation, all actions are atomic in the sense that they are always completed successfully without interruption. Consequently, while SPADL treats a pass as one action consisting of both the initiation and receival of the pass, Atomic-SPADL sees giving and receiving a pass as two separate actions. Because not all passes successfully reach a teammate, Atomic-SPADL introduces an interception action if the ball was intercepted by the other team or an out event if the ball went out of play. Atomic-SPADL similarly divides shots, freekicks, and corners into two separate actions. Practically, the effect is that this representation helps to distinguish the contribution of the player who initiates the action (e.g., gives the pass) and the player who completes the action (e.g., receives the pass)
<br>
The atomic SPADL format is conversion from the original SPADL format, so we do not need to rewrite the code for the Wyscout API in order to be able to use this. <br>
<br>
**Action Type**<br>
The action type attribute can have 22 possible values. These are pass, cross, throw-in, crossed free kick, short free kick, crossed corner, short corner, take-on, foul, tackle, interception, shot, penalty shot, free kick shot, keeper save, keeper claim, keeper punch, keeper pick-up, clearance, bad touch, dribble and goal kick. A detailed definition of each action type is available here.

**Result**<br>
The result attribute can either have the value success, to indicate that an action achieved it’s intended result; or the value fail, if this was not the case. An example of a successful action is a pass which reaches a teammate. An example of an unsuccessful action is a pass which goes over the sideline. Some action types can have special results. These are offside (for passes, corners and free-kicks), own goal (for shots), and yellow card and red card (for fouls).

**Body Part**<br>
The body part attribute can have 4 possible values. These are foot, head, other and none. For Wyscout, which does not distinguish between the head and other body parts a special body part head/other is used.

### The problem we need to solve : moving to v3

The old wyscout event format from version 2 of the API looks like this: 
```
{
    "tags": [
    {
    "id": 1802,
    "tag": {
    "label": "not accurate"
            }
        }
    ],
    "eventId": 8,
    "eventName": "Pass",
    "eventSec": 1.8496730000000001,
    "id": 663292348,
    "matchId": 2852835,
    "matchPeriod": "1H",
    "playerId": 21123,
    "positions": [
        {
        "x": 52,
        "y": 47
        },
        {
        "x": 60,
        "y": 32
        }
    ],
    "subEventId": 85,
    "subEventName": "Simple pass",
    "teamId": 3185
}

```
The new version three format is somewhat more complex:
```
{
    "id": 601919968,
    "matchId": -168770,
    "matchPeriod": "1H",
    "minute": 8,
    "second": 21,
    "matchTimestamp": "00:08:21.568",
    "videoTimestamp": "507.568215",
    "relatedEventId": 601919969,
    "type": {
    "primary": "pass",
    "secondary": [
        "back_pass"
    ]
    },
    "location": {
    "x": 42, 
    "y": 83 
    },
    "team": {
    "id": 964,
    "name": "Borussia Dortmund",
    "formation": "3-4-3"
    },
    "opponentTeam": {
    "id": 961,
    "name": "Bayern München",
    "formation": "4-2-3-1"
    },
    "player": {
    "id": 156709,
    "name": "T. Hazard",
    "position": "RWF"
    },
    "pass": {
    "accurate": true,
    "length": 9.34,
    "angle": 148,
    "recipient": {
        "id": 419254,
        "name": "A. Hakimi",
        "position": "RWB"
    },
    "endLocation": {
        "x": 34,
        "y": 89
    }
    },
    "shot": null,
    "groundDuel": null,
    "aerialDuel": null,
    "infraction": null,
    "carry": null,
    "possession": {
    "id": 601919966,
    "duration": "3.293842",
    "types": [
        "throw_in"
    ],
    "eventsNumber": 3,
    "eventIndex": 1,
    "startLocation": {
        "x": 29,
        "y": 100
    },
    "endLocation": {
        "x": 43,
        "y": 28
    },
    "team": {
        "id": 964,
        "name": "Borussia Dortmund",
        "formation": "3-4-3"
    },
    "attack": null
    }
}
```

There is no current conversion between the Wyscout v3 format and the SPADL data format, so we are going to build it ourselves.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
import pandas as pd
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.filterwarnings(action="ignore", message="credentials were not supplied. open data access only")
import tqdm

In [3]:
from socceraction.spadl.wyscout import *
import pandas as pd
from socceraction.data.wyscout import WyscoutLoader
from socceraction.data.base import _localloadjson
from socceraction.data.wyscout.loader import _convert_events
from socceraction.data.wyscout.schema import WyscoutEventSchema

The Wyscout loader code from the SoccerAction databse also looks to be broken, so I will use some of the methods in the file and the hyperlinks in the WyscoutLoader class to load the data manually in the below cell

In [4]:
# Events have been downloaded from the link in the PublicWyscoutLoader class - events="https://ndownloader.figshare.com/files/14464685",

# A modified version of the code from the events method of the PublicWyscoutLoader class
obj = _localloadjson("../Data/Events/v2_response.json")

In [5]:
obj

{'events': [{'id': 1646549577,
   'playerId': 372255,
   'teamId': 3161,
   'matchId': 5345057,
   'matchPeriod': '1H',
   'eventSec': 3.187,
   'eventId': 8,
   'eventName': 'Pass',
   'subEventId': 85,
   'subEventName': 'Simple pass',
   'positions': [{'x': 49, 'y': 50}, {'x': 34, 'y': 50}],
   'tags': [{'id': 1801}]},
  {'id': 1646549578,
   'playerId': 20635,
   'teamId': 3161,
   'matchId': 5345057,
   'matchPeriod': '1H',
   'eventSec': 6.645,
   'eventId': 8,
   'eventName': 'Pass',
   'subEventId': 83,
   'subEventName': 'High pass',
   'positions': [{'x': 34, 'y': 50}, {'x': 75, 'y': 96}],
   'tags': [{'id': 1801}]},
  {'id': 1646549581,
   'playerId': 330003,
   'teamId': 3161,
   'matchId': 5345057,
   'matchPeriod': '1H',
   'eventSec': 9.512,
   'eventId': 1,
   'eventName': 'Duel',
   'subEventId': 10,
   'subEventName': 'Air duel',
   'positions': [{'x': 75, 'y': 96}, {'x': 83, 'y': 100}],
   'tags': [{'id': 701}, {'id': 1802}]},
  {'id': 1646549855,
   'playerId': 3441

In [6]:
raw_df = pd.DataFrame(obj['events'])
raw_df

,id,playerId,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,subEventName,positions,tags
0,1646549577,372255,3161,5345057,1H,3.187,8,Pass,85,Simple pass,"[{'x': 49, 'y': 50}, {'x': 34, 'y': 50}]",[{'id': 1801}]
1,1646549578,20635,3161,5345057,1H,6.645,8,Pass,83,High pass,"[{'x': 34, 'y': 50}, {'x': 75, 'y': 96}]",[{'id': 1801}]
2,1646549581,330003,3161,5345057,1H,9.512,1,Duel,10,Air duel,"[{'x': 75, 'y': 96}, {'x': 83, 'y': 100}]","[{'id': 701}, {'id': 1802}]"
3,1646549855,344132,3157,5345057,1H,9.512,1,Duel,10,Air duel,"[{'x': 25, 'y': 4}, {'x': 17, 'y': 0}]","[{'id': 703}, {'id': 1801}]"
4,1646549582,0,3161,5345057,1H,11.790,5,Interruption,50,Ball out of the field,"[{'x': 73, 'y': 100}]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
1564,1646551402,0,3157,5345057,2H,2848.000,5,Interruption,50,Ball out of the field,"[{'x': 100, 'y': 5}]",[]
1565,1646551403,518231,3157,5345057,2H,2866.000,3,Free Kick,30,Corner,"[{'x': 100, 'y': 0}, {'x': 96, 'y': 45}]","[{'id': 801}, {'id': 1802}]"
1566,1646550872,7905,3161,5345057,2H,2867.000,7,Others on the ball,71,Clearance,"[{'x': 4, 'y': 55}, {'x': 38, 'y': 59}]","[{'id': 1401}, {'id': 1802}]"
1567,1646551404,291591,3157,5345057,2H,2872.000,8,Pass,85,Simple pass,"[{'x': 62, 'y': 41}, {'x': 67, 'y': 3}]",[{'id': 1802}]


In [7]:
raw_df.loc[raw_df.subEventId.isin([22, 23, 24, 26])]

,id,playerId,teamId,matchId,matchPeriod,eventSec,eventId,eventName,subEventId,subEventName,positions,tags
1316,1646551241,518231,3157,5345057,2H,1986.0,2,Foul,23,Out of game foul,"[{'x': 19, 'y': 28}]",[{'id': 1702}]
1318,1646550614,286223,3161,5345057,2H,1993.0,2,Foul,23,Out of game foul,"[{'x': 82, 'y': 64}]",[{'id': 1702}]
1356,1646551270,291591,3157,5345057,2H,2166.0,2,Foul,23,Out of game foul,"[{'x': 56, 'y': 49}]",[{'id': 1702}]
1357,1646550658,372255,3161,5345057,2H,2166.0,2,Foul,23,Out of game foul,"[{'x': 47, 'y': 49}]",[{'id': 1702}]


Just take a single match from this file for the rest of the investigation 

In [8]:
df_events = _convert_events(pd.DataFrame(raw_df))
events_df = cast(DataFrame[WyscoutEventSchema], df_events)

In [9]:
# check if there is a single row for each event ie - nothing has been exploded
len(events_df), events_df.event_id.nunique()

(1569, 1569)

The root function that does all the nice trickery we care about is the convert_to_actions function. Take a look at the output of this function with v2 and v3 event data from Wyscout and see what it returns.

*Note Im just using the first team id as home team here, may or may not be the home team but it saves downloading the Teams.json data*

In [10]:
spadl_actions = convert_to_actions(events_df, 1609)
spadl_actions

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id
0,5345057,1,3.187,3161,372255,53.55,34.00,69.30,34.00,1646549577,0,0,1,0
1,5345057,1,6.645,3161,20635,69.30,34.00,26.25,65.28,1646549578,0,0,1,1
2,5345057,1,9.512,3161,330003,26.25,65.28,76.65,0.00,1646549581,1,0,0,2
3,5345057,1,28.240,3161,330003,17.85,68.00,9.45,42.16,1646549583,2,2,1,3
4,5345057,1,31.440,3161,8327,9.45,42.16,9.45,42.16,1646549586,0,8,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,5345057,2,2843.000,3157,344132,38.85,14.28,14.70,31.28,1646551401,0,0,0,1137
1138,5345057,2,2844.000,3161,20635,90.30,36.72,0.00,0.00,1646550870,0,18,0,1138
1139,5345057,2,2866.000,3157,518231,0.00,0.00,4.20,30.60,1646551403,0,5,0,1139
1140,5345057,2,2867.000,3161,7905,100.80,37.40,39.90,27.88,1646550872,0,18,0,1140


Go through each component of the convert_to_actions function and find what will need adjusted for the v3 format

In [11]:
tags = get_tagsdf(events_df)
tags.columns

Index(['goal', 'own_goal', 'assist', 'key_pass', 'counter_attack', 'left_foot',
       'right_foot', 'head/body', 'direct', 'indirect', 'dangerous_ball_lost',
       'blocked', 'high', 'low', 'interception', 'clearance', 'opportunity',
       'feint', 'missed_ball', 'free_space_right', 'free_space_left',
       'take_on_left', 'take_on_right', 'sliding_tackle', 'anticipated',
       'anticipation', 'red_card', 'yellow_card', 'second_yellow_card',
       'position_goal_low_center', 'position_goal_low_right',
       'position_goal_mid_center', 'position_goal_mid_left',
       'position_goal_low_left', 'position_goal_mid_right',
       'position_goal_high_center', 'position_goal_high_left',
       'position_goal_high_right', 'position_out_low_right',
       'position_out_mid_left', 'position_out_low_left',
       'position_out_mid_right', 'position_out_high_center',
       'position_out_high_left', 'position_out_high_right',
       'position_post_low_right', 'position_post_mid_left',
    

The "tags" from the v2 of the Wyscout API look like some combination of the type.secondary column, the event attributes like shot, pass etc and possession_types from v3 

In [12]:
events_tagged = pd.concat([events_df, get_tagsdf(events_df)], axis=1)
events_tagged.head(3)

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate
0,1646549577,5345057,1,3187.0,3161,372255,8,Pass,85,Simple pass,"[{'x': 49, 'y': 50}, {'x': 34, 'y': 50}]",[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,1646549578,5345057,1,6645.0,3161,20635,8,Pass,83,High pass,"[{'x': 34, 'y': 50}, {'x': 75, 'y': 96}]",[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
2,1646549581,5345057,1,9512.0,3161,330003,1,Duel,10,Air duel,"[{'x': 75, 'y': 96}, {'x': 83, 'y': 100}]","[{'id': 701}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True


The make_new_positions function just extracts the two x,y coordinates into start and end coordinates, anything that doesnt have an end location is removed - for v3 this is done for us already, so just filter df where we have endLocation values for the event

In [13]:
converted_positions = make_new_positions(events_tagged)
converted_positions.head(3)

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate,start_x,start_y,end_x,end_y
0,1646549577,5345057,1,3187.0,3161,372255,8,Pass,85,Simple pass,[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,49,50,34,50
1,1646549578,5345057,1,6645.0,3161,20635,8,Pass,83,High pass,[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,34,50,75,96
2,1646549581,5345057,1,9512.0,3161,330003,1,Duel,10,Air duel,"[{'id': 701}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,75,96,83,100


The fix_wyscout_events function will create coordinates that are needed for the spadl format for events that it doesnt exist. <br>
It is made up of multiple functions, defined below.

In [14]:
# The function is made up of these
# events = create_shot_coordinates(events) # this should be able to remain the same for v3
# events = convert_duels(events) # this will need to be rewritten to use the groundDuel and aerialDuel fields in v3, but logic should remain the same
# events = insert_interception_passes(events) # dont think this is needed anymore
# events = add_offside_variable(events) # this can remain the same other than offside is now contained in the type.primary field
# convert_touches(events) # same as above
# convert_simulations(events) # same as above but the simulation field is the infraction type (might also be in type secondary)

# events = fix_wyscout_events(events)
# events.head(3)

The create_df_actions is the main function that we will need to focus on for the conversion to the new types. It is a manually defined process whereby the event event is assigned to one of the SciSports actions based on the types (type.primary and type.secondary in v3). The result of the event and body part is also assigned here.

In [15]:
# determine_bodypart_id(event) # The logic here will remain the same other than using the body_type field from wyscout v3
# determine_type_id(event)  # This is probably the bulk of the work - need to manually check the primary/secondary combos and decide which category they will fit into
# determine_result_id(event) # This should be simle and remain mostly the same

# actions = create_df_actions(events)
# actions.head(3)

### Start to convert the v3 data 

In [16]:
v3_json = _localloadjson("../Data/Events/v3_response.json")

v3_events = v3_json['events']

In [17]:
v3_df = pd.json_normalize(v3_events, sep='_')
v3_df.head(3)

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location
0,1646549577,5345057,1H,0,3,00:00:03.187,5.187274,1.646550e+09,NaN,NaN,NaN,NaN,NaN,pass,"[back_pass, short_or_medium_pass]",49.0,50.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,372255,L. Martínez,SS,True,180.0,None,16.0,20635.0,F. Acerbi,CB,34.0,50.0,1.646550e+09,6.324848,[],4.0,0.0,49.0,50.0,75.0,96.0,3161.0,Internazionale,3-5-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1646549578,5345057,1H,0,6,00:00:06.645,8.645442,1.646550e+09,NaN,NaN,NaN,NaN,NaN,pass,"[forward_pass, long_pass, pass_to_final_third,...",34.0,50.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,20635,F. Acerbi,CB,True,36.0,high,53.0,330003.0,D. Dumfries,RWB,75.0,96.0,1.646550e+09,6.324848,[],4.0,1.0,49.0,50.0,75.0,96.0,3161.0,Internazionale,3-5-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1646549581,5345057,1H,0,9,00:00:09.512,11.512122,1.646550e+09,NaN,NaN,NaN,NaN,NaN,duel,"[aerial_duel, loss]",75.0,96.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,330003,D. Dumfries,RWB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646550e+09,6.324848,[],4.0,2.0,49.0,50.0,75.0,96.0,3161.0,Internazionale,3-5-2,NaN,NaN,344132.0,Theo Hernández,LB,184.0,False,188.0,1.646550e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
v3_df.loc[v3_df.id==1646551241]

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location
1320,1646551241,5345057,2H,78,6,01:18:06.169,4892.169859,NaN,NaN,NaN,NaN,NaN,NaN,infraction,[yellow_card],19.0,28.0,3157,Milan,4-2-3-1,3161,Internazionale,3-5-2,518231,S. Tonali,LDMF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,out_of_play_foul,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
# The new format is only one row shorter than the old format
v3_df.shape

(1574, 94)

In [20]:
events_df.shape

(1569, 12)

In [21]:
v3_df.loc[~v3_df.id.isin(events_tagged.event_id.values)]

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location
475,1646551627,5345057,1H,28,57,00:28:57.106,1739.106043,1.646551e+09,NaN,NaN,NaN,NaN,NaN,duel,[aerial_duel],93.0,49.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646550e+09,14.0434785,"[corner, set_piece_attack, attack]",16.0,5.0,100.0,0.0,81.0,37.0,3161.0,Internazionale,3-5-2,NaN,NaN,134496.0,M. Maignan,GK,191.0,False,NaN,1.646551e+09,NaN,True,False,False,left,0.0094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
803,1646551631,5345057,2H,45,59,00:45:59.341,2965.341227,1.646550e+09,NaN,NaN,NaN,NaN,NaN,duel,"[dribble, ground_duel, offensive_duel]",76.0,20.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646550e+09,19.0069005,[attack],16.0,12.0,34.0,59.0,86.0,35.0,3161.0,Internazionale,3-5-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,False,left,0.0489,NaN,NaN,NaN,NaN,NaN,NaN,254493.0,R. Krunić,RDMF,dribble,True,False,None,None,False,left,1.646550e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1030,1646551632,5345057,2H,57,14,00:57:14.106,3640.106992,1.646550e+09,NaN,NaN,NaN,NaN,NaN,duel,[loose_ball_duel],89.0,41.0,3157,Milan,4-2-3-1,3161,Internazionale,3-5-2,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646550e+09,22.3040755,[attack],12.0,10.0,9.0,64.0,100.0,28.0,3157.0,Milan,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,right,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1156,1646551633,5345057,2H,66,2,01:06:02.497,4168.497713,1.646550e+09,NaN,NaN,NaN,NaN,NaN,duel,"[defensive_duel, ground_duel]",65.0,16.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,0,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.646551e+09,4.7890275,[throw_in],10.0,8.0,8.0,100.0,33.0,83.0,3157.0,Milan,4-2-3-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,404397.0,Brahim Diaz,AMF,defensive_duel,None,None,False,False,False,None,1.646551e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1348,1646551636,5345057,2H,79,57,01:19:57.700,5003.700632,1.646551e+09,NaN,NaN,NaN,NaN,NaN,duel,"[defensive_duel, ground_duel]",48.0,6.0,316

In [22]:
wyscout_periods = wyscout_periods = {"1H": 1, "2H": 2, "E1": 3, "E2": 4, "P": 5}

v3_df["period_id"] = v3_df.matchPeriod.apply(lambda x: wyscout_periods[x])
v3_df["milliseconds"] = (v3_df.second + (v3_df.minute * 60)) * 1000

In [23]:
# from socceraction.spadl.wyscout_v3 import convert_simulations as convert_simulations_v3

# df_events = convert_simulations_v3(df_events)

Test the rewritten make_new_positions function

In [24]:
from socceraction.spadl.wyscout_v3 import make_new_positions as make_new_positions_v3

updated_positions = make_new_positions_v3(v3_df)
updated_positions.head(3)

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [25]:
len(v3_df), len(updated_positions)

(1574, 1574)

In [26]:
from socceraction.spadl.wyscout_v3 import create_shot_coordinates as create_shot_coordinates_v3

shot_coords = create_shot_coordinates_v3(updated_positions)

In [27]:
shot_coords

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [28]:
from socceraction.spadl.wyscout import create_shot_coordinates

v2_shot_coords = create_shot_coordinates(converted_positions)

In [29]:
cols = ['start_x', 'start_y', 'end_x', 'end_y']
v2_shot_coords.loc[v2_shot_coords.type_name=='Shot'][cols].reset_index()

,index,start_x,start_y,end_x,end_y
0,67,67,67,100,50.0
1,118,78,52,100,40.0
2,198,88,53,100,55.0
3,392,86,61,86,61.0
4,398,82,49,100,60.0
5,484,81,37,100,60.0
6,494,79,34,79,34.0
7,605,94,35,100,60.0
8,622,92,43,100,50.0
9,643,83,54,100,40.0


In [30]:
updated_positions.loc[updated_positions.shot_isGoal.notna()][cols].reset_index()

,index,start_x,start_y,end_x,end_y
0,67,67.0,67.0,100.0,50.0
1,118,78.0,52.0,100.0,40.0
2,198,88.0,53.0,100.0,55.0
3,392,86.0,61.0,86.0,61.0
4,398,82.0,49.0,100.0,60.0
5,485,81.0,37.0,100.0,60.0
6,495,79.0,34.0,79.0,34.0
7,606,94.0,35.0,100.0,60.0
8,623,92.0,43.0,100.0,50.0
9,644,83.0,54.0,100.0,40.0


In [31]:
v2_shot_coords.loc[v2_shot_coords.type_name=='Shot'][cols].reset_index() == shot_coords.loc[shot_coords.shot_xg.notna()][cols].reset_index()

,index,start_x,start_y,end_x,end_y
0,True,True,True,True,True
1,True,True,True,True,True
2,True,True,True,True,True
3,True,True,True,True,True
4,True,True,True,True,True
5,False,True,True,True,True
6,False,True,True,True,True
7,False,True,True,True,True
8,False,True,True,True,True
9,False,True,True,True,True


In [32]:
from socceraction.spadl.wyscout_v3 import convert_duels as convert_duels_v3

converted_duels = convert_duels_v3(shot_coords)

In [33]:
from socceraction.spadl.wyscout import convert_duels

converted_duels_v2 = convert_duels(v2_shot_coords)

In [34]:
converted_duels_v2.loc[~converted_duels_v2.event_id.isin(converted_duels.id)]

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate,start_x,start_y,end_x,end_y


In [35]:
len(converted_duels.loc[~converted_duels.id.isin(converted_duels_v2.event_id)])

2

Some of the duel conversion doesnt quite match up due to the order of the actions being returned in a diferent order between v2 and v3. As far as I can see there is no way to sort this as the events have the same matchTimestamp. 

In [36]:
from socceraction.spadl.wyscout_v3 import insert_interception_passes as insert_interception_passes_v3

interception_passes_v3 = insert_interception_passes_v3(converted_duels)

In [37]:
interception_passes_v2 = insert_interception_passes(converted_duels_v2)

In [38]:
len(interception_passes_v3), len(interception_passes_v2)

(1226, 1224)

In [39]:
len(v3_df.loc[v3_df.type_primary=="interception"]), len(interception_passes_v3.loc[interception_passes_v3.type_primary=="interception"])

(65, 90)

In [40]:
interception_passes_v3.loc[interception_passes_v3.type_primary=="interception"].tail(30)

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [41]:
from socceraction.spadl.wyscout_v3 import convert_touches as convert_touches_v3

# converted_touches = convert_touches_v3(interception_passes_v3)
# converted_touches_v2 = convert_touches(interception_passes_v2)

The fix_wyscout_events method should now yield identical results (aside from the ordering of actions in convert duels)

In [42]:
from socceraction.spadl.wyscout_v3 import fix_wyscout_events as fix_wyscout_events_v3

fixed_df_v2 = fix_wyscout_events(converted_positions)

In [43]:
fixed_df = fix_wyscout_events_v3(v3_df)
fixed_df.loc[fixed_df.id==1646549616]

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,accurate,not_accurate,offside,same_loc,same_team,touch_other
38,1646549616,5345057,1H,3,44,00:03:44.785,226.785091,1.646550e+09,NaN,NaN,NaN,NaN,NaN,touch,[],48.0,12.0,3161,Internazionale,3-5-2,3157,Milan,4-2-3-1,239291,F. Dimarco,LWB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,224000,48.0,12.0,48.0,12.0,NaN,NaN,0,False,True,False


In [60]:
from socceraction.spadl.wyscout_v3 import add_offside_variable as add_offside_variable_v3

offside = add_offside_variable_v3(interception_passes_v3)

In [61]:
fixed_touches = convert_touches_v3(offside)
fixed_touches.loc[fixed_touches.id==1646549616]

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [62]:
events_df_v3 = make_new_positions_v3(v3_df)
fixed_df_v3 = fix_wyscout_events_v3(events_df_v3)

In [63]:
fixed_df_v3.loc[fixed_df_v3.id==1646549616]

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [64]:
from socceraction.spadl.wyscout_v3 import determine_bodypart_id as determine_bodypart_id_v3

fixed_df_v3["time_seconds"] = fixed_df_v3["milliseconds"] / 1000
fixed_df_v3 = fixed_df_v3.rename(columns={"matchId": "game_id"})
df_actions = fixed_df_v3[
    [
        "game_id",
        "period_id",
        "time_seconds",
        "team_id",
        "player_id",
        "start_x",
        "start_y",
        "end_x",
        "end_y",
    ]
].copy()
df_actions["original_event_id"] = fixed_df_v3["id"].astype(object)

In [65]:
df_actions["bodypart_id"] = fixed_df_v3.apply(determine_bodypart_id_v3, axis=1)

In [66]:
fixed_df_v2["time_seconds"] = fixed_df_v2["milliseconds"] / 1000
df_actions_v2 = fixed_df_v2[
    [
        "game_id",
        "period_id",
        "time_seconds",
        "team_id",
        "player_id",
        "start_x",
        "start_y",
        "end_x",
        "end_y",
    ]
].copy()
df_actions_v2["original_event_id"] = fixed_df_v2["event_id"].astype(object)
df_actions_v2["bodypart_id"] = fixed_df_v2.apply(determine_bodypart_id, axis=1)

The differences here are because I have added the left foot and right foot flag from shots which did not look like they were available in the v2. There is also some information missing from v2 that makes interceptions come through as foot even when they are with head - so our new version does a better job.

In [67]:
merged = df_actions.merge(df_actions_v2, on="original_event_id")
len(merged.loc[merged['bodypart_id_x'] != merged['bodypart_id_y']])

59

In [68]:
len(fixed_df_v3), len(fixed_df_v2)

(1226, 1223)

In [69]:
indeces = fixed_df_v2.loc[(fixed_df_v2["interception"]==True) & (fixed_df_v2["subtype_id"].isin([0, 10, 11, 12, 13, 72]))]

In [70]:
from socceraction.spadl.wyscout_v3 import determine_type_id as determine_type_id_v3

df_actions["type_id"] = fixed_df_v3.apply(determine_type_id_v3, axis=1)
df_actions_v2["type_id"] = fixed_df_v2.apply(determine_type_id, axis=1)

In [71]:
action_counts = df_actions.type_id.value_counts().reset_index()
action_counts_v2 = df_actions_v2.type_id.value_counts().reset_index()

compare = action_counts.merge(action_counts_v2, on='index', suffixes=['', '_v2'], how='outer').rename(columns={'index': "action_id"}).sort_values(by='action_id')
compare

,action_id,type_id,type_id_v2
0,0,727,734
7,1,27,27
4,2,34,34
15,3,4,4
5,4,34,34
12,5,6,6
16,6,2,2
6,7,29,29
3,8,37,37
14,9,4,4


In [72]:
# fixed_df_v3.accurate = fixed_df_v3.accurate.fillna(False)
# fixed_df_v3.not_accruate = fixed_df_v3.not_accurate.fillna(False)

### Accuracy flag mapping

In [75]:
from socceraction.spadl.wyscout_v3 import determine_result_id as determine_result_id_v3
df_actions["result_id"] = fixed_df_v3.apply(determine_result_id_v3, axis=1)
df_actions_v2["result_id"] = fixed_df_v2.apply(determine_result_id, axis=1)

In [76]:
result_counts = df_actions.result_id.value_counts().reset_index()
result_counts_v2 = df_actions_v2.result_id.value_counts().reset_index()

compare = result_counts.merge(result_counts_v2, on='index', suffixes=['', '_v2'], how='outer')
# .rename(columns={'index': "result_id"}).sort_values(by='result_id')
compare

,index,result_id,result_id_v2
0,1,1056,1041
1,0,169,181
2,2,1,1


In [94]:
from socceraction.spadl.wyscout_v3 import convert_to_actions as convert_to_actions_v3

v3_actions = convert_to_actions_v3(v3_df, 3161)
v3_actions

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id
0,5345057,1,3.0,3161,372255,51.45,34.00,35.70,34.00,1646549577,0,0,1,0
1,5345057,1,6.0,3161,20635,35.70,34.00,78.75,2.72,1646549578,0,0,1,1
2,5345057,1,9.0,3161,330003,78.75,2.72,28.35,68.00,1646549581,1,0,0,2
3,5345057,1,28.0,3161,330003,87.15,0.00,95.55,25.84,1646549583,2,2,1,3
4,5345057,1,31.0,3161,8327,95.55,25.84,95.55,25.84,1646549586,0,8,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,5345057,2,5540.0,3157,254493,44.10,35.36,38.85,14.28,1646551400,0,0,1,1137
1138,5345057,2,5542.0,3157,344132,38.85,14.28,14.70,31.28,1646551401,0,0,0,1138
1139,5345057,2,5566.0,3157,518231,0.00,0.00,4.20,30.60,1646551403,0,5,0,1139
1140,5345057,2,5569.0,3157,291591,4.20,30.60,39.90,27.88,NaN,0,21,1,1140


In [95]:
v2_actions = convert_to_actions(events_df, 3161)
v2_actions

,game_id,period_id,time_seconds,team_id,player_id,start_x,start_y,end_x,end_y,original_event_id,bodypart_id,type_id,result_id,action_id
0,5345057,1,3.187,3161,372255,51.45,34.00,35.70,34.00,1646549577,0,0,1,0
1,5345057,1,6.645,3161,20635,35.70,34.00,78.75,2.72,1646549578,0,0,1,1
2,5345057,1,9.512,3161,330003,78.75,2.72,28.35,68.00,1646549581,1,0,0,2
3,5345057,1,28.240,3161,330003,87.15,0.00,95.55,25.84,1646549583,2,2,1,3
4,5345057,1,31.440,3161,8327,95.55,25.84,95.55,25.84,1646549586,0,8,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1137,5345057,2,2843.000,3157,344132,38.85,14.28,14.70,31.28,1646551401,0,0,0,1137
1138,5345057,2,2844.000,3161,20635,14.70,31.28,0.00,0.00,1646550870,0,18,0,1138
1139,5345057,2,2866.000,3157,518231,0.00,0.00,4.20,30.60,1646551403,0,5,0,1139
1140,5345057,2,2867.000,3161,7905,4.20,30.60,39.90,27.88,1646550872,0,18,0,1140


### Map accurate events
Look at all the events that are marked as accurate in v2 and try to replicate in v3

In [197]:
# Get all accurate events from v2
v2_accurate = events_tagged.loc[events_tagged.accurate==True]
v2_accurate.head()

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,positions,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate
0,1646549577,5345057,1,3187.0,3161,372255,8,Pass,85,Simple pass,"[{'x': 49, 'y': 50}, {'x': 34, 'y': 50}]",[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1,1646549578,5345057,1,6645.0,3161,20635,8,Pass,83,High pass,"[{'x': 34, 'y': 50}, {'x': 75, 'y': 96}]",[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
3,1646549855,5345057,1,9512.0,3157,344132,1,Duel,10,Air duel,"[{'x': 25, 'y': 4}, {'x': 17, 'y': 0}]","[{'id': 703}, {'id': 1801}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False
5,1646549583,5345057,1,28240.0,3161,330003,3,Free Kick,36,Throw in,"[{'x': 83, 'y': 100}, {'x': 91, 'y': 62}]",[{'id': 1801}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
6,1646549584,5345057,1,29850.0,3161,8327,1,Duel,13,Ground loose ball duel,"[{'x': 91, 'y': 62}, {'x': 91, 'y': 64}]","[{'id': 702}, {'id': 1801}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False


Get the same events from v3 and inspect - do this using the updated_positions df as it has the related events joined

In [198]:
v3_should_be_accurate = updated_positions.loc[updated_positions.id.isin(v2_accurate.event_id.values)]
v3_should_be_accurate.head()

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

Map the 'easier to map' categories (those including an accurate flag) and see if they match the v2 flags perfectly

In [276]:
next_action = updated_positions.shift(-1)
suffix = '_next'

for column in updated_positions.columns:
    # Perform the shift operation and add the suffix
    shifted_column = updated_positions[column].rename(column + suffix)
    # Join the shifted column to the original dataframe
    updated_positions = pd.concat([updated_positions, shifted_column], axis=1)

In [277]:
updated_positions['accurate'] = False
updated_positions.loc[updated_positions.shot_onTarget == True, 'accurate'] = True
updated_positions.loc[updated_positions.pass_accurate == True, 'accurate'] = True
updated_positions.loc[updated_positions.type_secondary.apply(lambda x: "save" in x), 'accurate'] = True
updated_positions.loc[updated_positions.aerialDuel_firstTouch == True, 'accurate'] = True
updated_positions.loc[(updated_positions.groundDuel_keptPossession==True), 'accurate'] = True
updated_positions.loc[((updated_positions.groundDuel_stoppedProgress==True) | (updated_positions.groundDuel_recoveredPossession==True)), 'accurate'] = True

In [278]:
merged_df = pd.merge(updated_positions, events_tagged, right_on='event_id', left_on='id', suffixes=('_v3', '_v2'))
compare_result = merged_df['accurate_v3'] == merged_df['accurate_v2']
different_rows = merged_df[~compare_result]
different_rows

,id,matchId,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id_v3,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id_v3,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id_v3,milliseconds_v3,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_oppo

Split the remaining unmapped rows by type primary and see what the conversion to v2 is

In [279]:
different_rows.loc[different_rows.type_primary == "duel"][['accurate_v3']].value_counts()

accurate_v3
False          73
True           15
dtype: int64

In [282]:
different_rows.loc[different_rows.type_primary == "duel"][['type_primary', 'type_secondary', 'aerialDuel_firstTouch', 'groundDuel_duelType', 'groundDuel_stoppedProgress', 'groundDuel_progressedWithBall', 'groundDuel_keptPossession', 'groundDuel_recoveredPossession', 'carry_progression', 'type_primary_next', 'type_secondary_next', 'aerialDuel_firstTouch_next', 'groundDuel_duelType_next', 'groundDuel_stoppedProgress_next', 'groundDuel_progressedWithBall_next', 'groundDuel_keptPossession_next', 'groundDuel_recoveredPossession_next', 'carry_progression_next', 'type_primary_related_event', 'type_secondary_related_event', 'aerialDuel_firstTouch_related_event', 'groundDuel_duelType_related_event', 'groundDuel_stoppedProgress_related_event', 'groundDuel_progressedWithBall_related_event', 'groundDuel_keptPossession_related_event', 'groundDuel_recoveredPossession_related_event', 'carry_progression_related_event', 'accurate_v2', 'accurate_v3', 'team_id_v3', 'team_id_next', 'team_id_related_event']].tail(5)

,type_primary,type_secondary,aerialDuel_firstTouch,groundDuel_duelType,groundDuel_stoppedProgress,groundDuel_progressedWithBall,groundDuel_keptPossession,groundDuel_recoveredPossession,carry_progression,type_primary_next,type_secondary_next,aerialDuel_firstTouch_next,groundDuel_duelType_next,groundDuel_stoppedProgress_next,groundDuel_progressedWithBall_next,groundDuel_keptPossession_next,groundDuel_recoveredPossession_next,carry_progression_next,type_primary_related_event,type_secondary_related_event,aerialDuel_firstTouch_related_event,groundDuel_duelType_related_event,groundDuel_stoppedProgress_related_event,groundDuel_progressedWithBall_related_event,groundDuel_keptPossession_related_event,groundDuel_recoveredPossession_related_event,carry_progression_related_event,accurate_v2,accurate_v3,team_id_v3,team_id_next,team_id_related_event
1369,duel,"[defensive_duel, ground_duel]",NaN,defensive_duel,True,None,None,False,NaN,duel,"[defensive_duel, ground_duel]",NaN,defensive_duel,True,None,None,False,NaN,pass,"[cross, lateral_pass, loss, pass_to_penalty_area]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,3161,3161,3157.0
1374,duel,"[loose_ball_duel, carry]",NaN,NaN,NaN,NaN,NaN,NaN,6.338,duel,"[loose_ball_duel, carry]",NaN,NaN,NaN,NaN,NaN,NaN,6.338,duel,"[ground_duel, loss, offensive_duel]",NaN,offensive_duel,None,False,False,None,NaN,True,False,3161,3161,3161.0
1399,duel,"[loose_ball_duel, recovery, counterpressing_re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,duel,"[loose_ball_duel, recovery, counterpressing_re...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,"[forward_pass, short_or_medium_pass]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,3157,3157,3157.0
1404,duel,[loose_ball_duel],NaN,NaN,NaN,NaN,NaN,NaN,NaN,duel,[loose_ball_duel],NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,"[lateral_pass, long_pass]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,3157,3157,3157.0
1447,duel,"[loose_ball_duel, recovery]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,duel,"[loose_ball_duel, recovery]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,pass,"[forward_pass, short_or_medium_pass]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,3161,3161,3161.0


### Map non accurate events

Validate that the shots and passes are perfectly mapped to the v2 accuracy flags

In [202]:
len(v2_accurate.loc[v2_accurate.type_id==8]), len(v3_accurate_passes)

(635, 720)

In [115]:
df_events = create_shot_coordinates_v3(v3_sample)
df_events = convert_duels_v3(df_events)
df_events = insert_interception_passes_v3(df_events)
df_events = add_offside_variable_v3(df_events)
df_events = convert_touches_v3(df_events)

In [116]:
v2_sample = converted_positions.loc[converted_positions.event_id.isin(in_v3_and_not_v2.original_event_id.values)]
df_events_v2 = create_shot_coordinates(v2_sample)
df_events_v2 = convert_duels(df_events_v2)
df_events_v2 = insert_interception_passes(df_events_v2)
df_events_v2 = add_offside_variable(df_events_v2)
df_events_v2 = convert_touches(df_events_v2)

In [126]:
fixed_df_v3.not_accurate.value_counts()

True     30
False     8
Name: not_accurate, dtype: int64

In [119]:
df_events[['same_team', 'same_loc', 'touch_other', 'id', 'accurate', 'not_accurate']]

,same_team,same_loc,touch_other,id,accurate,not_accurate
0,False,False,True,1646549611,NaN,NaN
1,False,False,True,1646550509,NaN,NaN
2,False,False,False,1646549685,NaN,NaN
3,False,False,True,1646550676,NaN,NaN
4,True,False,False,1646549897,NaN,NaN
5,False,False,True,1646550060,NaN,NaN
6,True,False,False,1646549882,NaN,NaN
7,False,False,True,1646549942,NaN,NaN
8,True,False,False,1646550356,NaN,NaN
9,True,False,False,1646550380,NaN,NaN


In [120]:
df_events_v2.loc[df_events_v2.event_id.isin(df_events.id.values)][['same_team', 'same_loc', 'touch_other', 'accurate', 'not_accurate']]

,same_team,same_loc,touch_other,accurate,not_accurate
0,False,False,True,False,False
1,False,False,True,False,False
2,False,False,False,False,True
3,False,False,True,False,False
4,True,False,False,False,False
5,False,False,True,False,False
6,True,False,False,False,False
7,False,False,True,False,False
8,True,False,False,False,False
9,True,False,False,False,False


In [102]:
saves = fixed_df_v2.loc[fixed_df_v2.type_id==9]

In [103]:
fixed_df_v3.loc[fixed_df_v3.id.isin(saves.event_id.values)]

,id,game_id,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [96]:
subtypes = fixed_df_v2.loc[(fixed_df_v2.interception == True) & (fixed_df_v2.subtype_id!=71) & (fixed_df_v2["subtype_id"].isin([0, 10, 11, 12, 13, 72]))]

In [97]:
fixed_df_v3.loc[fixed_df_v3.id.isin(subtypes.event_id.values)]

,id,game_id,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

The change between clearance and interception from v2 to v3 looks to be a way street. A clearance with tag 1401 (interception), gets taken as type.primary of interception in v3, however, we keep no reference of that fact that it was also a clearance in v2, meaning we lose a large portion of the clearances from the dataset and increase the number of interceptions.

In [65]:
clearances_v2 = fixed_df_v2.loc[fixed_df_v2.subtype_id==71]
clearances_v2.head(4)
# not_matching = passes.loc[~passes.original_event_id.isin(df_actions.loc[df_actions.type_id==0].original_event_id.values)]
# fixed_df.loc[fixed_df.id.isin(not_matching.original_event_id) & (fixed_df.id==1646549616)][['type_primary', 'type_secondary', 'id']]

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate,start_x,start_y,end_x,end_y,offside,same_loc,time_seconds
119,1646549664,5345057,1,521300.0,3161,405597,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,3,35,21,60.0,0,False,521.3
135,1646549666,5345057,1,564100.0,3161,239291,7,Others on the ball,71,Clearance,[{'id': 1802}],False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,16,46,30,38.0,0,False,564.1
140,1646549671,5345057,1,577200.0,3161,21095,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,3,50,0,61.0,0,True,577.2
144,1646549673,5345057,1,631000.0,3161,8327,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1801}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,5,53,25,63.0,0,False,631.0


In [69]:
interceptions_v3 = fixed_df_v3.loc[fixed_df_v3.type_primary=="interception"]
interceptions_v3.head(4)

,id,game_id,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [70]:
v2_interceptions_marked_as_clearance = clearances_v2.loc[clearances_v2.event_id.isin(interceptions_v3.id.values)]
v2_interceptions_marked_as_clearance.head(4)

,event_id,game_id,period_id,milliseconds,team_id,player_id,type_id,type_name,subtype_id,subtype_name,tags,goal,own_goal,assist,key_pass,counter_attack,left_foot,right_foot,head/body,direct,indirect,dangerous_ball_lost,blocked,high,low,interception,clearance,opportunity,feint,missed_ball,free_space_right,free_space_left,take_on_left,take_on_right,sliding_tackle,anticipated,anticipation,red_card,yellow_card,second_yellow_card,position_goal_low_center,position_goal_low_right,position_goal_mid_center,position_goal_mid_left,position_goal_low_left,position_goal_mid_right,position_goal_high_center,position_goal_high_left,position_goal_high_right,position_out_low_right,position_out_mid_left,position_out_low_left,position_out_mid_right,position_out_high_center,position_out_high_left,position_out_high_right,position_post_low_right,position_post_mid_left,position_post_low_left,position_post_mid_right,position_post_high_center,position_post_high_left,position_post_high_right,through,fairplay,lost,neutral,won,accurate,not_accurate,start_x,start_y,end_x,end_y,offside,same_loc,time_seconds
119,1646549664,5345057,1,521300.0,3161,405597,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,3,35,21,60.0,0,False,521.3
140,1646549671,5345057,1,577200.0,3161,21095,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,3,50,0,61.0,0,True,577.2
144,1646549673,5345057,1,631000.0,3161,8327,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1801}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,5,53,25,63.0,0,False,631.0
190,1646549705,5345057,1,961100.0,3161,8327,7,Others on the ball,71,Clearance,"[{'id': 1401}, {'id': 1802}]",False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,6,45,48,9.0,0,False,961.1


In [71]:
interceptions_v3_marked_as_clearance = interceptions_v3.loc[interceptions_v3.id.isin(v2_interceptions_marked_as_clearance.event_id.values)]
interceptions_v3_marked_as_clearance.head()

,id,game_id,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id,milliseconds,start_x,start_y,end_x,end_y,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDuel_opponent_positio

In [75]:
v2v3 = fixed_df_v3.merge(fixed_df_v2, left_on='id', right_on='event_id')
v2v3.loc[v2v3.subtype_id==71].head(5)

,id,game_id_x,matchPeriod,minute,second,matchTimestamp,videoTimestamp,relatedEventId,shot,groundDuel,aerialDuel,infraction,carry,type_primary,type_secondary,location_x,location_y,team_id_x,team_name,team_formation,opponentTeam_id,opponentTeam_name,opponentTeam_formation,player_id_x,player_name,player_position,pass_accurate,pass_angle,pass_height,pass_length,pass_recipient_id,pass_recipient_name,pass_recipient_position,pass_endLocation_x,pass_endLocation_y,possession_id,possession_duration,possession_types,possession_eventsNumber,possession_eventIndex,possession_startLocation_x,possession_startLocation_y,possession_endLocation_x,possession_endLocation_y,possession_team_id,possession_team_name,possession_team_formation,possession_attack,pass,aerialDuel_opponent_id,aerialDuel_opponent_name,aerialDuel_opponent_position,aerialDuel_opponent_height,aerialDuel_firstTouch,aerialDuel_height,aerialDuel_relatedDuelId,possession,possession_attack_withShot,possession_attack_withShotOnGoal,possession_attack_withGoal,possession_attack_flank,possession_attack_xg,infraction_yellowCard,infraction_redCard,infraction_type,infraction_opponent_id,infraction_opponent_name,infraction_opponent_position,groundDuel_opponent_id,groundDuel_opponent_name,groundDuel_opponent_position,groundDuel_duelType,groundDuel_keptPossession,groundDuel_progressedWithBall,groundDuel_stoppedProgress,groundDuel_recoveredPossession,groundDuel_takeOn,groundDuel_side,groundDuel_relatedDuelId,carry_progression,carry_endLocation_x,carry_endLocation_y,shot_bodyPart,shot_isGoal,shot_onTarget,shot_goalZone,shot_xg,shot_postShotXg,shot_goalkeeperActionId,shot_goalkeeper,shot_goalkeeper_id,shot_goalkeeper_name,infraction_opponent,location,period_id_x,milliseconds_x,start_x_x,start_y_x,end_x_x,end_y_x,id_related_event,matchId_related_event,matchPeriod_related_event,minute_related_event,second_related_event,matchTimestamp_related_event,videoTimestamp_related_event,relatedEventId_related_event,shot_related_event,groundDuel_related_event,aerialDuel_related_event,infraction_related_event,carry_related_event,type_primary_related_event,type_secondary_related_event,location_x_related_event,location_y_related_event,team_id_related_event,team_name_related_event,team_formation_related_event,opponentTeam_id_related_event,opponentTeam_name_related_event,opponentTeam_formation_related_event,player_id_related_event,player_name_related_event,player_position_related_event,pass_accurate_related_event,pass_angle_related_event,pass_height_related_event,pass_length_related_event,pass_recipient_id_related_event,pass_recipient_name_related_event,pass_recipient_position_related_event,pass_endLocation_x_related_event,pass_endLocation_y_related_event,possession_id_related_event,possession_duration_related_event,possession_types_related_event,possession_eventsNumber_related_event,possession_eventIndex_related_event,possession_startLocation_x_related_event,possession_startLocation_y_related_event,possession_endLocation_x_related_event,possession_endLocation_y_related_event,possession_team_id_related_event,possession_team_name_related_event,possession_team_formation_related_event,possession_attack_related_event,pass_related_event,aerialDuel_opponent_id_related_event,aerialDuel_opponent_name_related_event,aerialDuel_opponent_position_related_event,aerialDuel_opponent_height_related_event,aerialDuel_firstTouch_related_event,aerialDuel_height_related_event,aerialDuel_relatedDuelId_related_event,possession_related_event,possession_attack_withShot_related_event,possession_attack_withShotOnGoal_related_event,possession_attack_withGoal_related_event,possession_attack_flank_related_event,possession_attack_xg_related_event,infraction_yellowCard_related_event,infraction_redCard_related_event,infraction_type_related_event,infraction_opponent_id_related_event,infraction_opponent_name_related_event,infraction_opponent_position_related_event,groundDuel_opponent_id_related_event,groundDuel_opponent_name_related_event,groundDue

In [132]:
df_events = create_shot_coordinates_v3(v3_df)
df_events = convert_duels_v3(df_events)
df_events = insert_interception_passes_v3(df_events)
# df_events = add_offside_variable(df_events)
# df_events = convert_touches(df_events)
# df_events = convert_simulations(df_events)

In [60]:
fixed_df_v3.loc[fixed_df_v3.id.isin(interception_clearances_v2.event_id.values)][['type_primary', 'type_secondary', 'groundDuel_duelType', 'start_x', 'end_x', 'start_y', 'end_y', 'team_id', 'team_id_related_event']].head(14)

,type_primary,type_secondary,groundDuel_duelType,start_x,end_x,start_y,end_y,team_id,team_id_related_event
119,interception,"[head_pass, loss]",NaN,3.0,79.0,35.0,40.0,3161,3157.0
140,interception,[head_pass],NaN,3.0,0.0,50.0,61.0,3161,3161.0
144,interception,"[head_pass, recovery, counterpressing_recovery]",NaN,5.0,25.0,53.0,63.0,3161,3161.0
191,interception,[head_pass],NaN,6.0,52.0,45.0,91.0,3161,3157.0
216,interception,[recovery],NaN,10.0,23.0,54.0,42.0,3161,3161.0
341,interception,[],NaN,9.0,100.0,51.0,64.0,3157,3161.0
351,interception,[loss],NaN,4.0,41.0,62.0,25.0,3157,3161.0
362,interception,"[head_pass, loss]",NaN,11.0,70.0,57.0,18.0,3161,3157.0
385,interception,"[head_pass, recovery, counterpressing_recovery]",NaN,9.0,18.0,61.0,47.0,3157,3157.0
503,interception,[],NaN,9.0,100.0,70.0,15.0,3157,3161.0
